In [62]:
# standard libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

# Data imputation
from sklearn.impute import SimpleImputer, KNNImputer

In [63]:
#Simple Imputers
imp_mean = SimpleImputer(strategy="mean")
imp_median = SimpleImputer(strategy="median")
imp_0 = SimpleImputer(strategy="constant", fill_value=0)

#KNN imputers
imp_knn5 = KNNImputer(n_neighbors=5, weights="distance")
imp_knn4 = KNNImputer(n_neighbors=4, weights="distance")
imp_knn3 = KNNImputer(n_neighbors=3, weights="distance")


In [64]:
# Import and display first five rows of advertising dataset
df = pd.read_csv('df_0723.csv')
df.head()

,COUNTRY,ID,READING_AVG,MATH_AVG,SCIENCE_AVG,GDP_2018,P_Enrollment_Totals,LS_completion,LS_completion_male,LS_completion_female,...,LS_national_assessment,P_annual_exp,LS_annual_exp,expect_degree,test_anxiety,study_habits,expect_good_grades,home_comp_internet,home_desk_quiet,home_schoolbooks
0,United States,USA,505.0,478,502,62605.592,24957955.89,98.75227,98.901003,98.606098,...,1.0,11727.023750,12693.449750,76.0,67.7,86.9,94.3,85.2,73.3,74.1
1,Russia,USSR-RUS,479.0,488,478,29266.855,6927980.00,99.41000,99.630000,99.170000,...,NaN,NaN,NaN,16.9,51.1,91.8,80.9,94.4,88.2,94.5
2,Belarus,USSR-BLR,474.0,472,471,20003.029,427752.00,99.93871,99.853840,100.000000,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Israel,ISR,470.0,463,462,37972.002,936214.00,98.07000,99.360000,96.930000,...,NaN,7971.310744,NaN,57.0,44.5,76.9,96.0,91.1,90.2,83.1
4,Ukraine,USSR-UKR,466.0,453,469,9283.433,1676550.00,99.45947,99.511510,99.417580,...,1.0,2612.067455,2511.505638,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# specify different types of columns
id_columns = ['COUNTRY','ID']
outcome_columns = ['READING_AVG','MATH_AVG','SCIENCE_AVG']
control_columns = ['GDP_2018']
driver_columns = np.setdiff1d(df.columns.tolist(),id_columns + outcome_columns + control_columns).tolist()

# Imputation

Possible Imputation Guidelines:

        1-10 missing  --> median --- won't change variance in data due to low missing vals
        11-22 missing --> knn (n = 5, weights = 'distance') --- ~50 vals available to impute from
        24-30 missing --> knn (n = 4, weights = 'distance') --- ~40 vals available to impute from
        31+ missing   --> knn (n = 3, weights = 'distance') --- ~30 vals available to impute from

In [88]:
# create a copy for imputation
imp_df = df.copy()

In [89]:
#columns grouped by suggested imputing methods above
median_imp =['READING_AVG','GDP_2018']
knn5_imp = ['LS_completion','LS_completion_male','LS_completion_female','LS_ta_presence', 'P_ta_presence','LS_guard_presence', 'P_sped_ta_presence','LS_sped_ta_presence','salary_per_instruction_hr']
knn4_imp = ['P_national_assessment','LS_national_assessment','P_annual_exp','LS_annual_exp']
knn3_imp = ['expect_degree','test_anxiety','study_habits','expect_good_grades','home_comp_internet','home_desk_quiet','home_schoolbooks','teacher_sent _societal_contribution','teacher_need_multicultural_pd','teacher_PD_12M_Value','principal_PD_12M_Value']

In [68]:
#running median imputation and adding to new df

med_imp = imp_median.fit_transform(df[median_imp])
imp_med = pd.DataFrame(med_imp, columns= median_imp)
imp_df[median_imp] = imp_med

I noticed with the KNN imputing method, only columns listed in transform( ) are used as neighbors. This is why, for every "formula" variable, I added the previous formula's contents (already imputed) plus the new list that will be the focus for imputation.

In [69]:
#based on formula 1 columns, impute kn 5 columns
formula1 = outcome_columns + control_columns+ knn5_imp
knn_imp5 = imp_knn5.fit_transform(imp_df[formula1])
imp_5 = pd.DataFrame(knn_imp5, columns= formula1)
imp_df[formula1] = imp_5

In [70]:
#based on formula 2 columns, impute kn 4 columns
formula2 = outcome_columns + control_columns+ knn5_imp + knn4_imp
knn_imp4 = imp_knn4.fit_transform(imp_df[formula2])
imp_4 = pd.DataFrame(knn_imp4, columns= formula2)
imp_df[formula2] = imp_4

In [71]:
#based on formula 3 columns, impute kn 3 columns
formula3 = outcome_columns + control_columns+ knn5_imp + knn4_imp + knn3_imp
knn_imp3 = imp_knn3.fit_transform(imp_df[formula3])
imp_3 = pd.DataFrame(knn_imp3, columns= formula3)
imp_df[formula3] = imp_3

# Comparing Previous Averages to Imputed Averages

In [87]:
df.describe()

,READING_AVG,MATH_AVG,SCIENCE_AVG,GDP_2018,P_Enrollment_Totals,LS_completion,LS_completion_male,LS_completion_female,teacher_sent _societal_contribution,teacher_need_multicultural_pd,...,LS_national_assessment,P_annual_exp,LS_annual_exp,expect_degree,test_anxiety,study_habits,expect_good_grades,home_comp_internet,home_desk_quiet,home_schoolbooks
count,73.000000,74.000000,74.000000,73.000000,6.700000e+01,57.000000,57.000000,57.000000,39.000000,39.000000,...,50.000000,51.000000,48.000000,39.000000,40.000000,40.00000,40.000000,43.000000,43.000000,43.000000
mean,452.027397,457.459459,456.945946,38152.755137,2.672815e+06,92.020157,93.126493,90.971811,89.958974,16.892308,...,0.600000,6845.181356,7570.797835,45.164103,57.025000,81.46000,83.975000,84.593023,82.916279,83.839535
std,53.168749,57.086138,52.042894,25845.824321,5.406551e+06,14.755536,14.106214,15.570077,7.326725,10.504971,...,0.494872,3698.441062,4631.277701,17.054101,11.666625,6.98404,10.250572,18.015343,12.500322,7.125445
min,340.000000,325.000000,336.000000,7304.503000,2.653200e+04,0.000000,0.000000,0.000000,65.600000,3.600000,...,0.000000,847.356358,847.356358,16.900000,33.500000,59.80000,60.600000,19.300000,43.200000,60.200000
25%,412.000000,414.750000,417.500000,19043.317000,2.001585e+05,91.193333,92.366667,90.016667,86.150000,10.300000,...,0.000000,3988.708022,3437.641245,33.750000,47.725000,78.12500,77.050000,86.200000,80.300000,80.850000
50%,466.000000,463.500000,466.000000,31938.663000,5.146640e+05,97.116667,97.816667,96.480000,92.300000,14.300000,...,1.000000,6877.313290,6853.625612,42.800000,55.800000,82.30000,84.300000,92.000000,88.200000,85.100000
75%,495.000000,499.750000,495.750000,52120.535000,2.795060e+06,98.876667,99.283333,98.980000,95.700000,20.800000,...,1.000000,8603.787121,10591.719512,57.700000,64.050000,86.95000,92.900000,95.300000,91.650000,88.300000
max,555.000000,591.000000,590.000000,130475.069000,2.942575e+07,100.000000,100.000000,100.000000,98.800000,45.900000,...,1.000000,20892.093910,21123.618500,76.300000,81.200000,91.80000,97.700000,98.200000,93.200000,95.300000


In [75]:
imp_df.describe()

,READING_AVG,MATH_AVG,SCIENCE_AVG,GDP_2018,P_Enrollment_Totals,LS_completion,LS_completion_male,LS_completion_female,teacher_sent _societal_contribution,teacher_need_multicultural_pd,...,LS_national_assessment,P_annual_exp,LS_annual_exp,expect_degree,test_anxiety,study_habits,expect_good_grades,home_comp_internet,home_desk_quiet,home_schoolbooks
count,74.000000,74.000000,74.000000,74.000000,6.700000e+01,74.000000,74.000000,74.000000,74.000000,74.000000,...,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000
mean,452.216216,457.459459,456.945946,38068.780919,2.672815e+06,92.732094,93.837486,91.691703,91.903585,20.210328,...,0.627190,6390.105450,7705.686900,48.239311,60.339155,82.916624,86.754258,76.628234,77.583955,82.405378
std,52.828300,57.086138,52.042894,25678.350464,5.406551e+06,13.245104,12.558288,14.097315,6.230617,12.350390,...,0.425206,3832.447780,4914.158757,15.140701,12.348996,6.020553,9.432825,23.274991,15.653052,7.434984
min,340.000000,325.000000,336.000000,7304.503000,2.653200e+04,0.000000,0.000000,0.000000,65.600000,3.600000,...,0.000000,847.356358,847.356358,16.900000,33.500000,59.800000,60.600000,19.300000,43.200000,60.200000
25%,412.250000,414.750000,417.500000,19151.608750,2.001585e+05,92.594701,92.865833,92.157681,89.055368,10.550521,...,0.000000,3125.528937,3205.790739,38.209846,51.525000,80.126189,80.039979,58.391431,65.177449,80.039245
50%,466.000000,463.500000,466.000000,31938.663000,5.146640e+05,96.960162,97.763034,96.433333,92.867824,14.600000,...,0.843054,6068.171296,6681.537449,47.643827,58.267824,84.188758,88.258049,90.530127,84.168453,84.493127
75%,494.500000,499.750000,495.750000,51503.195250,2.795060e+06,98.720000,99.080251,98.754399,97.105882,26.838911,...,1.000000,8609.214456,12056.536514,63.048462,71.875000,87.175000,96.140825,94.350000,90.473534,86.575000
max,555.000000,591.000000,590.000000,130475.069000,2.942575e+07,100.000000,100.000000,100.000000,98.800000,45.900000,...,1.000000,20892.093910,21123.618500,76.300000,81.200000,91.800000,97.700000,98.200000,93.200000,95.300000


In [85]:
imp_df.to_csv('imputed.csv', index = False)